In [61]:
import numpy as np
import matplotlib.pyplot as plt

In [73]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 1, 3, 0, 5, 10, 1, 2],
              [500, 700, 750, 600, 1450,
               800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2, 
               1, 3, 3, 1, 2]], dtype = np.float64)
y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype = np.float64)

In [74]:
def calc_std_feat(x):
  res = (x - x.mean()) / x.std()
  return res

In [75]:
def sigmoid(z):
  res = 1 / (1 + np.exp(-z))
  return res

In [76]:
X_st = X.copy()
X_st[2, :] = calc_std_feat(X[2, :])

#### 1*. Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.  

In [77]:
def calc_logloss(y, y_pred):
  #err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
  err = - np.mean(y * np.log(y_pred + 0e-10) + (1.0 - y) * np.log(1.0 - y_pred + 0e-10))
  err = np.sum(err)
  return err

#### 2. Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.

In [78]:
def eval_model(X, y, iterations, alpha=1e-4, output=True):
  np.random.seed(42)
  W = np.random.randn(X.shape[0])
  n = X.shape[1]
  for i in range(1, iterations+1):
    z = np.dot(W, X)
    y_pred = sigmoid(z)
    err = calc_logloss(y, y_pred)
    W -= alpha * (1/n * np.dot((y_pred - y), X.T))
    if output and i % (iterations / 10) == 0:
      print(i, W, err)
  return W

In [94]:
W = eval_model(X_st, y, iterations=1000000, alpha=1e-2)

100000 [-11.27192801  -1.45338517  -2.38300258   9.49385018] 0.2523828537754709
200000 [-16.2422902   -1.86937288  -3.90462901  13.50867797] 0.208744448215138
300000 [-20.12046178  -2.20554145  -5.06338174  16.65513544] 0.18225133394128198
400000 [-23.32957684  -2.48701096  -6.00950463  19.2580126 ] 0.16415892687401007
500000 [-26.06999299  -2.72865738  -6.81029352  21.47844134] 0.1509957371068139
600000 [-28.46471856  -2.94024139  -7.50574487  23.41628013] 0.14096415738168344
700000 [-30.59526535  -3.12845003  -8.12178098  25.13788906] 0.13303809324891214
800000 [-32.51817754  -3.29802355  -8.67608723  26.68935959] 0.1265923758182375
900000 [-34.27409432  -3.4524232   -9.18120823  28.10384248] 0.1212260427861223
1000000 [-35.89309329  -3.59424249  -9.64632889  29.40588541] 0.11667083203751412


#### 3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 
#### (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).

In [107]:
def calc_pred_proba(W, X):
    z = np.dot(W, X)
    y_pred_proba = sigmoid(z)
    return y_pred_proba

In [108]:
calc_pred_proba(W, X_st)

array([0.34705581, 0.00984674, 1.        , 0.06777614, 0.93622567,
       0.04716302, 1.        , 0.01783199, 0.58968717, 0.99999976])

#### 4. Создайте функцию calc_pred, возвращающую предсказанный класс 
#### (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).

In [112]:
def calc_pred(W, X, threshold=0.6):
    y_pred = calc_pred_proba(W, X)
    y_pred = y_pred >= threshold
    return y_pred.astype(np.int)    

In [113]:
y.astype(np.int)

array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1])

In [114]:
y_pred = calc_pred(W, X_st)
y_pred

array([0, 0, 1, 0, 1, 0, 1, 0, 0, 1])

#### 5. Посчитайте Accuracy, матрицу ошибок, точность и полноту, а также F1 score.

In [120]:
accuracy = 100.0 - np.mean(np.abs(y - y_pred)*100)
accuracy

90.0

In [127]:
TP = 0
FP = 0
TN = 0
FN = 0
for i in range(len(y)):
    TP += y[i] == 1 and y[i] == y_pred[i]
    FP += y[i] == 1 and y[i] != y_pred[i]
    TN += y[i] == 0 and y[i] == y_pred[i]
    FN += y[i] == 0 and y[i] != y_pred[i]

m = [[TP, FP],
    [FN, TN]]
m

[[4, 1], [0, 5]]

In [128]:
precision = TP/(TP + FP)
precision

0.8

In [129]:
recall = TP/(TP + FN)
recall

1.0

In [130]:
f1 = 2*(precision * recall)/ (precision + recall)
f1

0.888888888888889

#### 6. Могла ли модель переобучиться? Почему?

Модель явно переобучилась (чтобы получить расхождение между предсказанием и целевыми значениями пришлось увеличить порог).
А вот с причиной затрудняюсь ответить.

#### 7*. Создайте функции eval_model_l1 и eval_model_l2 с применением L1 и L2 регуляризаций соответственно.

In [131]:
def eval_model_l1(X, y, iterations, alpha=1e-4, lambda_=1e-8, output=True):
  np.random.seed(42)
  W = np.random.randn(X.shape[0])
  n = X.shape[1]
  for i in range(1, iterations+1):
    z = np.dot(W, X)
    y_pred = sigmoid(z)
    err = calc_logloss(y, y_pred)
    W -= alpha * (1/n * np.dot((y_pred - y), X.T) + lambda_/2 * np.sign(W))
    if output and i % (iterations / 10) == 0:
      print(i, W, err)
  return W

In [132]:
W = eval_model_l1(X_st, y, iterations=100000, alpha=1e-2, lambda_=1e-8)

10000 [-2.77079445 -0.99580919  0.56650762  3.2676586 ] 0.4058305488604188
20000 [-4.59029834 -1.03598164 -0.13800859  4.4445785 ] 0.35332196239840574
30000 [-5.90941758 -1.08771186 -0.62104872  5.36110973] 0.3250055687987807
40000 [-6.96073435 -1.14368099 -0.98673709  6.13012758] 0.3066199108449885
50000 [-7.85440893 -1.19932056 -1.28729595  6.80520771] 0.29312188419455787
60000 [-8.6467636  -1.2534398  -1.54768563  7.41639777] 0.2823916664924978
70000 [-9.3691363  -1.30582496 -1.78113675  7.98151401] 0.2734026939028285
80000 [-10.04023602  -1.35653101  -1.99529671   8.51167308] 0.26560115240960896
90000 [-10.6718874   -1.40567686  -2.1948915    9.0141372 ] 0.2586623434147555
100000 [-11.27192398  -1.45338483  -2.3830013    9.49384694] 0.2523828951080822


In [133]:
def eval_model_l2(X, y, iterations, alpha=1e-4, lambda_=1e-8, output=True):
  np.random.seed(42)
  W = np.random.randn(X.shape[0])
  n = X.shape[1]
  for i in range(1, iterations+1):
    z = np.dot(W, X)
    y_pred = sigmoid(z)
    err = calc_logloss(y, y_pred)
    W -= alpha * (1/n * np.dot((y_pred - y), X.T) + lambda_ * W)
    if output and i % (iterations / 10) == 0:
      print(i, W, err)
  return W

In [134]:
W = eval_model_l2(X_st, y, iterations=100000, alpha=1e-2, lambda_=1e-8)

10000 [-2.77079339 -0.99580901  0.56650774  3.26765766] 0.4058305866139458
20000 [-4.59029464 -1.03598131 -0.1380076   4.44457565] 0.35332205315300214
30000 [-5.90941046 -1.08771128 -0.62104663  5.36110422] 0.32500570611383406
40000 [-6.96072295 -1.1436801  -0.98673361  6.1301187 ] 0.30662009560998743
50000 [-7.85439251 -1.19931927 -1.28729088  6.80519479] 0.2931221189105076
60000 [-8.64674143 -1.25343804 -1.54767879  7.41638019] 0.2823919542354091
70000 [-9.36910772 -1.30582265 -1.78112794  7.98149121] 0.27340303770376095
80000 [-10.04020042  -1.35652809  -1.99528575   8.51164453] 0.26560155496837135
90000 [-10.67184419  -1.40567327  -2.19487824   9.01410243] 0.25866280698505245
100000 [-11.27187263  -1.45338054  -2.38298558   9.4938055 ] 0.25238342147027537
